In [1]:
pip install -e .

Obtaining file:///home/natalie/nsalab/Users/naw1/desk2/dl/gym-teen
  Attempting uninstall: gym-maze
    Found existing installation: gym-maze 0.4
    Uninstalling gym-maze-0.4:
      Successfully uninstalled gym-maze-0.4
  Running setup.py develop for gym-maze
Note: you may need to restart the kernel to use updated packages.


In [2]:
import gym
import gym_teen
import numpy as np
import time
import sys
import math
import random

In [3]:
def get_explore_rate(t,min_explore_rate,decay_fact):
    return max(min_explore_rate, min(0.8, 1.0 - math.log10((t+1)/decay_fact)))


def get_learning_rate(t,min_learning_rate,decay_fact):
    return max(min_learning_rate, min(0.8, 1.0 - math.log10((t+1)/decay_fact)))

In [4]:
# variables: 
neps = 20
tstep = 200
min_explore_rate = 0.001
min_learning_rate = 0.2
#grid_size = tuple((env.observation_space.high + np.ones(env.observation_space.shape)).astype(int))
#decay_fact = np.prod(grid_size, dtype=float) / 10.0
decay_fact = np.prod((405,405),dtype=float)/10.0
#state_bounds = list(zip(env.observation_space.low, env.observation_space.high))
explore_rate = get_explore_rate(0,min_explore_rate,decay_fact)
learning_rate = get_learning_rate(0,min_learning_rate,decay_fact)

In [5]:
max_t = np.prod((405,405),dtype=int)*100
streak_to_end = 100
solved_t = np.prod((405,405),dtype=int)


In [6]:
def select_action(env,state, q_table,explore_rate):
    # Select a random action
    if random.random() < explore_rate:
        action = env.action_space.sample()
    # Select the action with the highest q
    else:
        action = int(np.argmax(q_table[state]))
    return action

In [7]:
def state_to_bucket(state,env):
    grid_size = tuple((env.observation_space.high + np.ones(env.observation_space.shape)).astype(int))
    state_bounds = list(zip(env.observation_space.low, env.observation_space.high))
    bucket_indice = []
    for i in range(len(state)):
        if state[i] <= state_bounds[i][0]:
            bucket_index = 0
        elif state[i] >= state_bounds[i][1]:
            bucket_index = grid_size[i] - 1
        else:
            # Mapping the state bounds to the bucket array
            bound_width = state_bounds[i][1] - state_bounds[i][0]
            offset = (grid_size[i]-1)*state_bounds[i][0]/bound_width
            scaling = (grid_size[i]-1)/bound_width
            bucket_index = int(round(scaling*state[i] - offset))
        bucket_indice.append(bucket_index)
    return tuple(bucket_indice)

In [8]:
def run_qlearn(env,q_table, explore_rate, learning_rate, neps=200,tstep=100):
    env.render()
    num_streaks = 0
    discount_factor = 0.99
    max_t = np.prod((405,405),dtype=int)*100
    streak_to_end = 100
    solved_t = np.prod((405,405),dtype=int)
    decay_fact = np.prod((405,405),dtype=float)/10.0
    
    for ep in range(neps):
        if ep == 0:
            time.sleep(1)
        obv = env.reset()
        print('------------'+str(ep))
        state_0 = state_to_bucket(obv,env)
        total_reward = 0
        
        for t in range(tstep):
            # select best action and execute
            action = select_action(env,state_0, q_table, explore_rate)
            try: 
                obv,reward,done,_ = env.step(action)
            except(IndexError):
                print(action)
                print(state_0)
                break
            
            # update the state you are in, add reward (observe result)
            state = state_to_bucket(obv,env)
            total_reward += reward
            #print(reward)
            #print('cur' + str(reward))
            #print(total_reward)
            #print(reward, total_reward)
            
            # update Q-table
            best_q = np.amax(q_table[state])
            q_table[state_0 + (action,)] += learning_rate * (reward + discount_factor * (best_q) - q_table[state_0 + (action,)])
            
            # update state
            state_0 = state
            
            env.render()
            #time.sleep(0.0000001)
            
            
            # update parameters
            #explore_rate = get_explore_rate(ep,min_explore_rate,decay_fact)
            #learning_rate = get_learning_rate(ep,min_learning_rate,decay_fact)
            
            if done:
                time.sleep(1)
                print("Episode %d finished after %f time steps with total reward = %f (streak %d)."
                      % (ep, t, total_reward, num_streaks))

                if t <= solved_t:
                    num_streaks += 1
                else:
                    num_streaks = 0
                break

            elif t >= max_t - 1:
                print("Episode %d timed out at %d with total reward = %f."
                      % (ep, t, total_reward))

        # It's considered done when it's solved over 120 times consecutively
        if num_streaks > streak_to_end:
            break
            
        explore_rate = get_explore_rate(ep,0.001,decay_fact)
        learning_rate = get_learning_rate(ep,0.2,decay_fact)
                
    return q_table

In [9]:
q_table0 = np.zeros((405,405) + (4,), dtype=float)

In [10]:

fin_q_table = run_qlearn(gym.make('maze-v0',height=60,width=60),q_table0,explore_rate,learning_rate,neps=100,tstep=max_t)

------------0
122.7725
120.41194444444444
118.95722222222223
117.875
117.00555555555556
113.9636111111111
108.89305555555555
110.28666666666666
108.89305555555555
103.73611111111111
98.66722222222222
99.9775
101.09583333333333
96.04888888888888
101.09583333333333
106.34611111111111
111.80416666666666
106.34611111111111
101.09583333333333
106.34611111111111
101.09583333333333
99.9775
94.99833333333333
96.04888888888888
91.04611111111112
87.14027777777778
86.93611111111112
90.02944444444445
89.04694444444445
86.78333333333333
86.4475
86.71666666666667
85.98055555555555
82.54305555555555
75.29638888888888
82.54305555555555
75.29638888888888
79.17527777777778
79.17527777777778
84.83972222222222
79.17527777777778
75.29638888888888
75.29638888888888
70.65277777777777
75.29638888888888
82.54305555555555
79.18027777777777
82.54305555555555
85.98055555555555
82.54305555555555
75.29638888888888
79.17527777777778
84.83972222222222
86.71027777777778
86.7475
86.75111111111111
86.7475
86.19861111111

93.07472222222222
98.06944444444444
93.07472222222222
92.7286111111111
97.73
103.2725
102.63527777777777
101.09583333333333
102.63527777777777
97.185
92.18027777777777
92.7286111111111
92.18027777777777
97.185
102.63527777777777
101.09583333333333
106.34611111111111
107.99527777777777
102.63527777777777
103.2725
97.73
92.7286111111111
93.07472222222222
92.7286111111111
93.07472222222222
88.09416666666667
87.89194444444445
86.76444444444445
86.75111111111111
86.73194444444445
87.14027777777778
86.93611111111112
90.02944444444445
89.04694444444445
90.02944444444445
89.04694444444445
93.68055555555556
89.04694444444445
90.02944444444445
91.04611111111112
96.04888888888888
94.99833333333333
90.02944444444445
94.99833333333333
90.02944444444445
86.93611111111112
87.14027777777778
87.68583333333333
92.18027777777777
87.68583333333333
86.75111111111111
87.68583333333333
87.14027777777778
87.68583333333333
86.75111111111111
86.7475
86.71027777777778
86.73194444444445
87.14027777777778
91.04611

121.20472222222222
119.45361111111112
118.03555555555556
119.45361111111112
121.20472222222222
119.45361111111112
120.82694444444445
122.76472222222222
123.98611111111111
122.76472222222222
120.82694444444445
122.76472222222222
123.61888888888889
124.75361111111111
123.98611111111111
124.75361111111111
125.43472222222222
124.75361111111111
123.98611111111111
124.75361111111111
125.48055555555555
124.82333333333334
123.98611111111111
124.75361111111111
125.43472222222222
125.66666666666667
125.65138888888889
125.66666666666667
125.43472222222222
125.66666666666667
125.48055555555555
124.82333333333334
125.48055555555555
125.64972222222222
125.48055555555555
124.82333333333334
123.76305555555555
122.7725
121.20472222222222
122.7725
120.41194444444444
121.51583333333333
119.81055555555555
121.51583333333333
119.81055555555555
118.79027777777777
117.82861111111112
118.79027777777777
119.81055555555555
118.79027777777777
117.82861111111112
118.66583333333334
119.64611111111111
120.76
121.98

117.17527777777778
118.68027777777777
119.39333333333333
118.70611111111111
117.17527777777778
115.67361111111111
111.64416666666666
106.31916666666666
96.94388888888889
104.38888888888889
111.64416666666666
115.67361111111111
111.64416666666666
104.38888888888889
110.86194444444445
110.86194444444445
115.74194444444444
110.86194444444445
115.67361111111111
117.17527777777778
118.70611111111111
118.67166666666667
118.63666666666667
118.40416666666667
118.67055555555555
119.0911111111111
121.33277777777778
123.60694444444445
123.93722222222222
124.03111111111112
124.11777777777777
123.20083333333334
122.4636111111111
122.54777777777778
120.77583333333334
121.47833333333334
120.04861111111111
119.57138888888889
121.49055555555556
122.00333333333333
120.04861111111111
119.01416666666667
120.04861111111111
122.00333333333333
123.96583333333334
122.00333333333333
121.49055555555556
122.00333333333333
123.15805555555555
124.19611111111111
124.86833333333334
125.01638888888888
123.43083333333

73.72277777777778
77.91
77.91
84.16777777777777
77.91
77.91
81.18222222222222
83.59527777777778
83.59527777777778
85.27111111111111
85.27111111111111
86.84361111111112
86.70861111111111
83.59527777777778
86.70861111111111
83.59527777777778
86.70861111111111
86.84361111111112
86.85944444444445
86.8411111111111
86.85944444444445
86.84361111111112
86.70861111111111
86.86888888888889
86.88194444444444
86.86888888888889
86.85944444444445
86.83222222222223
86.82583333333334
86.82888888888888
86.82583333333334
86.83222222222223
86.85944444444445
86.84361111111112
86.70861111111111
86.86888888888889
86.88194444444444
86.86888888888889
86.88194444444444
85.87555555555555
86.70861111111111
86.84361111111112
85.27111111111111
86.84361111111112
86.82583333333334
86.83222222222223
86.82583333333334
86.84361111111112
85.27111111111111
86.84361111111112
86.85944444444445
86.84361111111112
85.27111111111111
85.27111111111111
86.28277777777778
85.27111111111111
85.27111111111111
83.59527777777778
81.18

...

124.06111111111112
126.46388888888889
126.37583333333333
129.3322222222222
129.51444444444445
126.46388888888889
129.51444444444445
129.3322222222222
126.37583333333333
126.46388888888889
126.37583333333333
126.46388888888889
124.06111111111112
124.3375
123.40222222222222
120.505
120.52111111111111
113.97194444444445
120.52111111111111
119.79388888888889
119.4563888888889
121.67972222222222
123.60666666666667
124.88361111111111
122.6025
124.88361111111111
125.54194444444444
124.88361111111111
122.6025
119.79388888888889
113.88861111111112
113.97194444444445
120.52111111111111
119.79388888888889
113.88861111111112
113.57472222222222
107.94722222222222
102.4786111111111
107.94722222222222
108.20666666666666
102.665
97.085
97.02194444444444
91.96111111111111
87.06416666666667
87.07444444444444
92.03972222222222
91.96111111111111
97.02194444444444
102.64777777777778
102.39777777777778
101.98472222222222
102.39777777777778
108.0775
107.66277777777778
108.0775
108.26861111111111
113.97194444

65.00555555555556
55.51444444444444
48.42861111111111
55.51444444444444
65.00555555555556
71.94583333333334
62.57388888888889
55.51444444444444
62.57388888888889
53.0875
76.91666666666667
86.01694444444445
83.5525
83.5525
86.01694444444445
86.01694444444445
83.5525
68.3025
83.5525
68.3025
68.3025
87.65222222222222
86.72055555555555
87.65222222222222
88.18333333333334
86.72777777777777
86.75166666666667
86.72777777777777
86.75166666666667
86.74638888888889
86.57027777777778
83.72694444444444
76.09666666666666
85.84416666666667
83.94222222222223
91.67722222222223
91.96111111111111
87.06416666666667
91.96111111111111
87.06416666666667
86.8463888888889
86.85861111111112
86.8463888888889
86.86583333333333
86.89
86.78972222222222
86.89
86.90833333333333
87.09611111111111
91.51361111111112
91.91527777777777
92.03972222222222
97.085
96.95111111111112
91.91527777777777
92.03972222222222
87.07444444444444
92.03972222222222
87.07444444444444
86.86583333333333
86.8463888888889
87.06416666666667
87

84.81805555555556
82.48638888888888
85.9336111111111
86.69333333333333
84.81805555555556
86.20416666666667
86.74694444444444
86.20416666666667
82.2611111111111
82.2611111111111
79.29027777777777
84.81805555555556
86.20416666666667
82.2611111111111
86.20416666666667
84.81805555555556
86.69333333333333
85.9336111111111
86.69333333333333
86.73083333333334
86.69333333333333
86.74694444444444
86.69333333333333
85.9336111111111
86.69333333333333
86.73083333333334
86.72166666666666
85.9336111111111
84.08694444444444
85.9336111111111
86.69333333333333
86.74694444444444
86.74833333333333
86.765
88.23944444444444
93.13638888888889
93.51638888888888
98.51472222222222
98.13583333333334
93.13638888888889
92.48416666666667
87.89611111111111
88.23944444444444
87.89611111111111
86.74833333333333
87.89611111111111
86.74833333333333
86.74694444444444
86.20416666666667
82.2611111111111
84.375
82.2611111111111
79.29027777777777
75.37416666666667
82.48638888888888
85.9336111111111
84.08694444444444
79.0588

117.39083333333333
116.09277777777778
117.39083333333333
117.76277777777777
117.48777777777778
117.36166666666666
117.48777777777778
117.36166666666666
117.48777777777778
117.76277777777777
118.68805555555555
117.76277777777777
117.39083333333333
116.09277777777778
112.49388888888889
107.38222222222223
105.54
107.38222222222223
102.29027777777777
97.24444444444444
102.29027777777777
107.38222222222223
105.54
107.38222222222223
105.54
100.45833333333333
95.44083333333333
90.69222222222223
89.21527777777777
85.56166666666667
82.30388888888889
85.56166666666667
89.21527777777777
85.56166666666667
82.30388888888889
77.47583333333333
82.30388888888889
85.25055555555555
86.57027777777778
88.27166666666666
87.4575
88.27166666666666
86.57027777777778
84.9
82.00361111111111
77.48194444444445
70.43944444444445
62.50111111111111
53.30166666666667
45.36055555555556
53.30166666666667
45.36055555555556
51.85333333333333
51.85333333333333
58.868611111111115
65.13138888888889
74.39361111111111
79.0588

KeyboardInterrupt: 

In [1]:
import gym
import gym_teen
import numpy as np
import time
import sys
import math
import random

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Convolution2D
from tensorflow.keras.optimizers import Adam

from matplotlib import pyplot as plt

from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [2]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [3]:
# Dont run this right away. Run it if you get an error 
del model

NameError: name 'model' is not defined

In [4]:
env =gym.make('maze-v0',height=60,width=60)
states = env.observation_space.shape[0]
actions = env.action_space.n

pygame 2.1.2 (SDL 2.0.18, Python 3.7.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
model = build_model(states, actions)

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 2)                 0         
                                                                 
 dense (Dense)               (None, 24)                72        
                                                                 
 dense_1 (Dense)             (None, 24)                600       
                                                                 
 dense_2 (Dense)             (None, 4)                 100       
                                                                 
Total params: 772
Trainable params: 772
Non-trainable params: 0
_________________________________________________________________


In [7]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [8]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    9/10000 [..............................] - ETA: 1:08 - reward: -53.2434

C:\Users\asong18\Anaconda3\envs\deeplearning3.7\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
C:\Users\asong18\Anaconda3\envs\deeplearning3.7\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 124s 12ms/step - reward: -92.8293
2 episodes - episode_reward: -121059.532 [-192833.551, -49285.514] - loss: 1380.044 - mae: 1644.418 - mean_q: -2162.682

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 122s 12ms/step - reward: -77.9368
2 episodes - episode_reward: -624442.552 [-980949.062, -267936.041] - loss: 347074.656 - mae: 6442.932 - mean_q: 1795.818

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 125s 13ms/step - reward: -94.1459
1 episodes - episode_reward: -244315.745 [-244315.745, -244315.745] - loss: 11057750.000 - mae: 79905.805 - mean_q: 108028.086

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 129s 13ms/step - reward: -86.4152
4 episodes - episode_reward: -342997.903 [-948515.833, -52975.383] - loss: 23222066.000 - mae: 125944.805 - mean_q: 169476.266

Interval 5 (40000 steps performed)
10000/10000 [=======================

In [9]:
scores = dqn.test(env, nb_episodes=10, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 10 episodes ...
Episode 1: reward: -986390.344, steps: 10000
Episode 2: reward: -867564.159, steps: 10000
